In [1]:
import json
import requests
import time
import pandas as pd
import csv

In [2]:
def read_api_key(file_name="api_key.txt"):
    try:
        with open(file_name, "r") as file:
            api_key = file.read().strip()
        return api_key
    except FileNotFoundError:
        print(f"Error: {file_name} not found.")
        return None
    except Exception as e:
        print(f"An error occurred while reading the API key: {e}")
        return None

In [3]:
api_key = read_api_key()
if api_key:
    print("API key successfully loaded.")
else:
    print("Failed to load API key.")

API key successfully loaded.


In [4]:
def load_list_from_csv(file_name):
    with open(file_name, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        return [row[0] for row in reader]

In [5]:
match_ids = load_list_from_csv('match_id_grandmaster_20241223_1403.csv')

In [6]:
REQUEST_DELAY = 1.2

In [7]:
def fetch_data_with_retries(url, max_retries=3, delay=REQUEST_DELAY):
    retries = 0
    while retries < max_retries:
        try:
            response = requests.get(url)
            if response.status_code == 200:
                return response.json()
            elif response.status_code == 429:
                print("Rate limit exceeded. Retrying...")
                time.sleep(delay * 2)
            else:
                print(f"Failed request to {url}. Status code: {response.status_code}")
                return None
        except requests.RequestException as e:
            print(f"Request error: {e}")
        retries += 1
        time.sleep(delay)
    return None

In [8]:
match_logs = []

for match_id in match_ids:
    match_url = f'https://asia.api.riotgames.com/tft/match/v1/matches/{match_id}?api_key={api_key}'
    match_data = fetch_data_with_retries(match_url)
    
    if match_data:
        match_logs.append(match_data)
    else:
        print(f"Failed to fetch data for match ID: {match_id}")
    
    time.sleep(REQUEST_DELAY)

print(f"Total matches retrieved: {len(match_logs)}")

Failed request to https://asia.api.riotgames.com/tft/match/v1/matches/KR_7433140694?api_key=RGAPI-28142bf7-3646-4db9-8ca9-4754f26fbcd3. Status code: 403
Failed to fetch data for match ID: KR_7433140694
Failed request to https://asia.api.riotgames.com/tft/match/v1/matches/KR_7427418173?api_key=RGAPI-28142bf7-3646-4db9-8ca9-4754f26fbcd3. Status code: 403
Failed to fetch data for match ID: KR_7427418173
Failed request to https://asia.api.riotgames.com/tft/match/v1/matches/KR_7423119502?api_key=RGAPI-28142bf7-3646-4db9-8ca9-4754f26fbcd3. Status code: 403
Failed to fetch data for match ID: KR_7423119502
Failed request to https://asia.api.riotgames.com/tft/match/v1/matches/KR_7429376104?api_key=RGAPI-28142bf7-3646-4db9-8ca9-4754f26fbcd3. Status code: 403
Failed to fetch data for match ID: KR_7429376104
Failed request to https://asia.api.riotgames.com/tft/match/v1/matches/KR_7417335207?api_key=RGAPI-28142bf7-3646-4db9-8ca9-4754f26fbcd3. Status code: 403
Failed to fetch data for match ID: KR_7

In [9]:
def analyze_tft_matches(matches_data):
    results = []

    def get_cost_display(rarity):
        rarity_to_cost = {
            0: 1, 1: 2, 2: 3, 4: 4, 6: 5, 8: 6
        }
        return rarity_to_cost.get(rarity, None)

    def check_items_in_unit(unit):
        healing_reduction_items = {
            'TFT_Item_Morellonomicon',
            'TFT_Item_RedBuff',
            'TFT_Item_SunlightCape',
            'TFT_Item_MoreMorellonomicon',
            'TFT_Item_CrestOfCinders',
            'TFT_Item_TheEternalFlame'
        }

        armor_reduction_items = {
            'TFT_Item_LastWhisper',
            'TFT_Item_Evenshroud',
            'TFT_Item_Equinox',
            'TFT_Item_EternalWhisper',
            'TFT_Item_ObsidianCleaver'
        }

        magic_reduction_items = {
            'TFT_Item_StatikkShiv',
            'TFT_Item_IonicSpark',
            'TFT_Item_StatikksFavor',
            'TFT_Item_CovalentSpark',
            'TFT_Item_ObsidianCleaver'
        }

        unit_items = set(unit.get('itemNames', []))

        healing_reduction = bool(unit_items & healing_reduction_items)
        armor_reduction = bool(unit_items & armor_reduction_items)
        magic_reduction = bool(unit_items & magic_reduction_items)

        return healing_reduction, armor_reduction, magic_reduction

    for match_data in matches_data:
        match_id = match_data['metadata']['match_id']

        participants = sorted(match_data['info']['participants'], 
                              key=lambda x: x['placement'])

        for participant in participants:
            placement = participant['placement']
            username = participant['riotIdGameName']
            damage = participant['total_damage_to_players']
            last_round = participant.get('last_round', 'Unknown')

            units = participant['units']
            unit_details = []
            total_cost = 0

            has_healing_reduction = False
            has_armor_reduction = False
            has_magic_reduction = False

            three_star_1cost = 0
            three_star_2cost = 0
            three_star_3cost = 0

            for unit in units:
                character = unit['character_id'].replace('TFT13_', '').replace('tft13_', '')
                tier = unit['tier']
                rarity = unit['rarity']
                cost = get_cost_display(rarity)

                if cost is not None:
                    unit_cost = (3 ** (tier - 1)) * cost
                    total_cost += unit_cost
                    unit_details.append(f"{character}({tier}★)({cost}cost)")
                else:
                    unit_details.append(f"{character}(SpecialUnit)")

                if tier == 3:
                    if cost == 1:
                        three_star_1cost += 1
                    elif cost == 2:
                        three_star_2cost += 1
                    elif cost == 3:
                        three_star_3cost += 1

                unit_healing_red, unit_armor_red, unit_magic_red = check_items_in_unit(unit)
                has_healing_reduction = has_healing_reduction or unit_healing_red
                has_armor_reduction = has_armor_reduction or unit_armor_red
                has_magic_reduction = has_magic_reduction or unit_magic_red

            total_three_star_units = three_star_1cost + three_star_2cost + three_star_3cost

            deck_type = "리롤" if total_three_star_units >= 1 else "운영"

            has_armor_or_magic_reduction = has_armor_reduction or has_magic_reduction

            results.append({
                '순위': placement,
                '유저명': username,
                '가한피해량': damage,
                '마지막 라운드': last_round,
                '사용한유닛': unit_details,
                '총합비용': total_cost,
                '매치ID': match_id,
                '상처': has_healing_reduction,
                '파열': has_armor_reduction,
                '파쇄': has_magic_reduction,
                '파열or파쇄': has_armor_or_magic_reduction,
                '1cost 3★': three_star_1cost,
                '2cost 3★': three_star_2cost,
                '3cost 3★': three_star_3cost,
                '덱타입': deck_type
            })

    return pd.DataFrame(results)

In [10]:
match_details = analyze_tft_matches(match_logs)

In [11]:
valid_match_ids = match_details.groupby('매치ID')['순위'].apply(lambda x: set(range(1, 9)) <= set(x)).reset_index()
valid_match_ids = valid_match_ids[valid_match_ids['순위']].drop('순위', axis=1)
match_details = pd.merge(match_details, valid_match_ids, on='매치ID', how='inner')

In [12]:
match_details.to_csv("match_details_grandmaster_20241223_1403.csv", index=False)